In [1]:
function bid!(Q,deal,database_state,database_target,data_state_all_bid,epsilon_greedy)
    bidding_hist=zeros(36,1);
    instance=(deal[1],deal[2],bidding_hist);
    for t in 1:turn_depth
        if t==1
            current_bid=1;
            state=atype(reshape(instance[(t-1)%2+1],(52)));
        else
            b=findlast(x->x==1,bidding_hist);
            current_bid=b[1];
            state=atype(reshape(vcat(instance[(t-1)%2+1],instance[3]),(88)));  
        end
        #Algorithm P
        target=determine_target(deepcopy(deal[3][:,(t-1)%2+1]),current_bid);
        #Data for experience replay
        push!(database_state[t],state);
        push!(database_target[t],target);
        #exploration
        action=explore(Q[t],state,current_bid,epsilon_greedy);
        # current bid is pass----- 
        if action== current_bid
            break;
        end
            
        #Update bidding history
        bidding_hist[action]=1;
    end   
        push!(data_state_all_bid,(deal[1],deal[2],bidding_hist))

    return 
end

function bid_to_the_end(Q,deal_set,number_of_deals)
    database_state=[[],[],[],[],[],[]];
    database_target=[[],[],[],[],[],[]];
    for i in 1:number_of_deals
        bid!(Q,deal_set[rand(1:end)],database_state,database_target,[],0);
    end 
    return database_state, database_target ,data_state_all_bid
end

bid_to_the_end (generic function with 1 method)

In [2]:
function bid(Q,deal;debug=false)
    bidding_hist=zeros(36,1);
    instance=(deal[1],deal[2],bidding_hist);
    for t in 1:turn_depth
        if debug
            println(string(t," bid"))
        end
        if t==1
            current_bid=1;
            state=atype(reshape(instance[(t-1)%2+1],(52)));
        else
            b=findlast(x->x==1,bidding_hist);
            current_bid=b[1];
            state=atype(reshape(vcat(instance[(t-1)%2+1],instance[3]),(88)));  
        end
        #Algorithm P
        target=determine_target(deepcopy(deal[3][:,(t-1)%2+1]),current_bid,debugging=debug);
        #Data for experience replay
        #exploration
        action=explore(Q[t],state,current_bid,0,debugging=debug);
        # current bid is pass----- 
        if action== current_bid
            break;
        end
            
        #Update bidding history
        bidding_hist[action]=1;
    end   

    return deal[1],deal[2],bidding_hist
end

bid (generic function with 1 method)